In [7]:
from gensim.models import Word2Vec
import json
import pandas as pd
import nltk

In [6]:
with open('pos_reviews.json') as data_file:    
    data_pos = json.load(data_file)
with open('neg_reviews.json') as data_file:    
    data_neg = json.load(data_file)

pos_texts = data_pos.keys()
neg_texts = data_neg.keys()
all_texts = pos_texts + neg_texts

N_POS_TEXTS = len(pos_texts)
N_NEG_TEXTS = len(neg_texts)
target = [1]*N_POS_TEXTS + [0]*N_NEG_TEXTS

In [8]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle') # потребуется в дальнейшем для разбивки текстов на предложения

In [11]:
def review_to_sentences(review, tokenizer):
    # разбиваем review на предложения. Возвращаем список предложений. Каждое предложение - список слов
    #
    # 1. NLTK Tokenizer требуется для того, чтобы разбить текст на предложения. Разбиваем на предложения
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. идем по каждому предложению
    sentences = []
    for raw_sentence in raw_sentences:
        # если предолжение пустое - пропускаем его
        if len(raw_sentence) > 0:
            # иначе запускаем review_to_wordlist и добавляем в sentences
            sentences.append(raw_sentence.split())
            
    # возвращаем предложения
    return sentences

In [12]:
%%time
sentences = []  # Initialize an empty list of sentences
print("Parsing sentences from training set")
for review in all_texts:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set
CPU times: user 3.2 s, sys: 109 ms, total: 3.31 s
Wall time: 3.3 s


In [15]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [16]:
%%time
# Initialize and train the model (this will take some time)
print("Training model...")
model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

Training model...
CPU times: user 27.3 s, sys: 2.52 s, total: 29.8 s
Wall time: 11 s


In [17]:
# если мы не собираемся больше перетренировывать модель - лучше сохранить ее закэшировать
model.init_sims(replace=True)

In [22]:
model.doesnt_match("ma".split())

'table'